# 02 - Regional Literature: *Indian Monsoon Onset and Withdrawal*

In [1]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping, Point
import datetime
import numpy as np
import rioxarray
from scipy.interpolate import griddata

import xarray as xr

from mpl_toolkits.axes_grid1 import make_axes_locatable

from visualization.custom_plots import standard_format, day_of_year_to_date, standard_format_single, plot_seasons_bk_results

In [2]:
# Geopackages with boundaries
country_boundary_file   = '../data/raw/external/GADM41_IND.gpkg' 
world_boundary_file     = '../data/raw/external/GADM41_WORLD.gpkg' 

# Path for results
results_path_image  = f'../data/results/images/literature_reference_sates.png'

# Geodataframes for boudaries - Level 0 is for country boundaries 
country_boundary    = gpd.read_file(country_boundary_file, layer = 'ADM_ADM_1')
world_boundary      = gpd.read_file(world_boundary_file, layer = 'ADM_0')

## The Indian Summer Monsoon

The term Monsoon probably derives from the Arabic word mausim or the Malayan monsin which both mean season [Zhisheng et al., 2015], and  it is traditionally associated with the rainy period which accompanies a change in the seasonal prevailing wind in much of the tropics. 

Areas prone to GM are identified by the IPCC as those in which the annual precipitation range (i.e. the difference between the annual maximum and minimum precipitation) exceeds 2.5 ${mm}/{day}$ , with no further requirements. However, this could
lead to the inclusion of areas where the source of precipitation is not monsoonal. Therefore, a subsequent analysis was performed by the IPCC based on the published literature [IPCC, 2021a].

<img src="../images/IPCC_AR6_WGI_AV_Figure_1.png" width="700">


The South-Asian Monsoon (SAM) – the part of the monsoon system which mostly influences the analyzed region – can be regarded as part of the Asian Summer Monsoon (ASM).


The onset of the Monsoon can be divided in three phases:

- 1st phase: onset in thesouth-eastern part of the Bay of Bengal (known as the BOB monsoon), associated with an overturning of the meridional air temperature gradient [Mao and Wu, 2007] and the development of the so-called “monsoon onset vortex”.  

- 2nd phase: onset over the South China Sea, driven by atmospheric internal variability combined with the thermal and mechanical effects due to orography. 

- 3rd phase: The Indian Summer Monsoon (ISM) onset (3rd phase) can be seen as the northward seasonal movement of the Intertropical Convergence Zone (ITCZ) [Gadgil, 2003], or alternatively, as the westward propagation of the BOB monsoon.

It is possible to have a look at this pattern directly from ERA5 precipitation data:

In [3]:
# -- ERA5 PRECIPITATION DATA
# NetCDF files
precipitation_data_path     = '../data/preprocessed/ERA5/total_precipitation/*.nc'

# Importing as xarray dataset
precipitation_data = xr.open_mfdataset(precipitation_data_path)

# Converting in mm/day
precipitation_data = precipitation_data.tp.load() * 24 * 1000

# Computing monthly average daily precipitation
precipitation_data_monthly = precipitation_data.groupby('time.month').mean()

# Renaming for better image display
precipitation_data_monthly = precipitation_data_monthly.rename('Total precipitation [mm/day]')

/home/jgrassi/miniconda3/lib/python3.11/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.24.2
  warnings.warn(


In [4]:
# Plotting
plot = precipitation_data_monthly.plot.contourf(col='month', col_wrap=3, figsize=(8,9), add_colorbar = False, vmin=0, vmax=4, levels=9)

# Applying the standard format function for better display of xarray plot
standard_format( plot,
                 country_boundary   = country_boundary,
                 # world_boundary     = world_boundary,
                 custom_cbar        = True
                )

## The Indian Meteorological Departmente (IMD)

The Indian Meteorological Department (IMD) is the primary national agency responsible for meteorological observations, weather forecasting, and seismology in India. Established in 1875, IMD operates under the Ministry of Earth Sciences. One of its key areas of focus is the study of the monsoon, a crucial weather phenomenon for India's agriculture and economy. IMD meticulously monitors and predicts the dates of onset and withdrawal of the monsoon, providing critical data and forecasts to aid in agricultural planning, water resource management, and disaster preparedness. 

In [ ]:
gridded_dates_file      = '../data/preprocessed/external/monsoon_dates.nc'
ds_rast = xr.open_dataset(gridded_dates_file)

ds_rast

<xarray.Dataset>
Dimensions:      (lon: 28, lat: 24, cluster: 2)
Coordinates:
  * lon          (lon) float64 69.0 70.0 71.0 72.0 73.0 ... 93.0 94.0 95.0 96.0
  * lat          (lat) float64 9.0 10.0 11.0 12.0 13.0 ... 29.0 30.0 31.0 32.0
  * cluster      (cluster) int64 0 1
Data variables:
    spatial_ref  int64 ...
    date         (lat, lon, cluster) float64 ...
Attributes:
    Description:  Interpolated onset and withdrawal dates of the monsoon seas...
    History:      1. Country boundaries were loaded from GADM41_IND.gpkg.\n2....

In [ ]:
# plot_seasons_bk_results( ds_rast.date, figsize=(10,5), cmaps=['summer', 'plasma_r'],
#                          lims=[[150,180],[245,300]], titles=['Monsoon Onset - Literature', 'Monsoon Withdrawal - Literature'],
#                          country_boundary=country_boundary, world_boundary=world_boundary)

# plt.savefig(results_path_image, dpi=300, bbox_inches='tight', transparent=False)